# Projet MRO - COP

Dans ce NoteBook, on s'applique à trouver des solutions pour notre modélisation sur le problème d'attribution de fréquences.

## Bibliothèques

In [1]:
import json
from itertools import product
from numpy import abs

## Initialisation de la modélisation

### Chargement des données

In [2]:
data='test.json'
max_bound = 1000000
nb_fct_cout = 0
fqc_max = 728 +1

### Chargement des données et tableau utiles à l'écriture des contraintes

In [3]:
with open(data, 'r') as f:
    f = json.load(f)
    stations=f['stations']
    liaisons = f['liaisons']
    interferences = f['interferences']
    regions = f['regions']
    nb_stations = len(stations)

taille_domaine = (2 * nb_stations-1) * f'{fqc_max} '+ f'{fqc_max}\n'


### Écriture des contraintes de notre modélisation

#### Contraintes sur les fréquences (par extension, contraintes fortes)

On impose les fréquences des domaines de chaque antenne (émettrice et réceptrice) pour chaque station comme indiqué dans le fichier des données

In [5]:
contraintes_fqc = f''
for station in stations:
    fqc_e = station['emetteur']
    fqc_r = station['recepteur']
    nb_tuple_e = 0
    nb_tuple_r = 0
    contraintes_e = f''
    contraintes_r = f''
    for fqc in fqc_e:
        contraintes_e += f'{fqc} 0\n' 
        nb_tuple_e+=1
    for fqc in fqc_r:
        contraintes_r += f'{fqc} 0\n' 
        nb_tuple_r+=1

    contraintes_fqc += f'1 {station["num"]} {max_bound} {nb_tuple_e}\n' + contraintes_e
    contraintes_fqc += f'1 {station["num"]+nb_stations} {max_bound} {nb_tuple_r}\n' + contraintes_r
    nb_fct_cout+=2

#### Contraintes sur les $\delta$ (par extension, contraintes fortes)

On vérifie que $|E_i - R_i|$ soit supérieur ou égal à $\delta$. Du fait de la valeur absolue, on rajoute dans les cas par défaut que deux fréquences $f_i$ et $f_j$, qui respectent par exemple, $|E_i - R_i|$, soient inclus pour $E_i = f_i$ et $R_i = f_j$ mais aussi $E_i = f_j$ et $R_i=f_i$

In [7]:
delta = stations[0]["delta"]
contraintes_delta = f''
for station in stations:
    contrainte_delta = f''
    fqc_e = station['emetteur']
    fqc_r = station['recepteur']
    fqc_possible = list(product(fqc_e, fqc_r))
    nb_tuple_delta = 0
    for item in fqc_possible:
        if abs(item[0]-item[1]) == station["delta"]:
            contrainte_delta += f'{item[0]} {item[1]} 0\n'
            contrainte_delta += f'{item[1]} {item[0]} 0\n'
            nb_tuple_delta +=2
    contraintes_delta += f'2 {station["num"]} {station["num"] + nb_stations} {max_bound} {nb_tuple_delta}\n' + contrainte_delta
    nb_fct_cout+=1



#### Contraintes sur les $\Delta$ (par extension, contraintes molles)

On vérifie que $|E_i - R_j|, |E_i - E_j|, |R_i - R_j| et |R_i - E_j|$ soient supérieurs ou égaux à $\Delta$. Du fait de la valeur absolue, on rajoute dans les cas par défaut que deux fréqeunces $f_i$ et $f_j$, qui respectent par exemple et sans perte de généralité, $|E_i - R_j|$, soient inclus pour $E_i = f_i$ et $R_j = f_j$ mais aussi $E_i = f_j$ et $R_j=f_i$

In [9]:
contraintes_Delta = f''
for interference in interferences:
    
    fqc_e_x = stations[interference["x"]]['emetteur']
    fqc_r_x = stations[interference["x"]]['recepteur']
    fqc_e_y = stations[interference["y"]]['emetteur']
    fqc_r_y = stations[interference["y"]]['recepteur']

    fqc_possible_e_x_e_y = list(product(fqc_e_x, fqc_e_y))
    fqc_possible_e_x_r_y = list(product(fqc_e_x, fqc_r_y))
    fqc_possible_r_x_r_y = list(product(fqc_r_x, fqc_r_y))
    fqc_possible_r_x_e_y = list(product(fqc_r_x, fqc_e_y))

    contraintes_e_x_e_y = f''
    nb_tuple_delta_e_x_e_y = 0
    for item in fqc_possible:
        if abs(item[0]-item[1]) >= interference["Delta"]:
            contraintes_e_x_e_y += f'{item[0]} {item[1]} 0\n'
            contraintes_e_x_e_y += f'{item[1]} {item[0]} 0\n'
            nb_tuple_delta_e_x_e_y +=2   

    contraintes_e_x_r_y = f''
    nb_tuple_delta_e_x_r_y = 0
    for item in fqc_possible:
        if abs(item[0]-item[1]) >= interference["Delta"]:
            contraintes_e_x_r_y += f'{item[0]} {item[1]} 0\n'
            contraintes_e_x_r_y += f'{item[1]} {item[0]} 0\n'
            nb_tuple_delta_e_x_r_y +=2  

    
    contraintes_r_x_r_y = f''
    nb_tuple_delta_r_x_r_y = 0
    for item in fqc_possible:
        if abs(item[0]-item[1]) >= interference["Delta"]:
            contraintes_r_x_r_y += f'{item[0]} {item[1]} 0\n'
            contraintes_r_x_r_y += f'{item[1]} {item[0]} 0\n'
            nb_tuple_delta_r_x_r_y +=2 

    contraintes_r_x_e_y = f''
    nb_tuple_delta_r_x_e_y = 0
    for item in fqc_possible:
        if abs(item[0]-item[1]) >= interference["Delta"]:
            contraintes_r_x_e_y += f'{item[0]} {item[1]} 0\n'
            contraintes_r_x_e_y += f'{item[1]} {item[0]} 0\n'
            nb_tuple_delta_r_x_e_y +=2  

    contrainte_Delta_e_x_e_y = f'2 {interference["x"]} {interference["y"]} {interference["Delta"]} {nb_tuple_delta_e_x_e_y}\n' + contraintes_e_x_e_y
    contrainte_Delta_e_x_r_y = f'2 {interference["x"]} {interference["y"]+nb_stations} {interference["Delta"]} {nb_tuple_delta_e_x_r_y}\n' + contraintes_e_x_r_y
    contrainte_Delta_r_x_e_y = f'2 {interference["x"]+nb_stations} {interference["y"]} {interference["Delta"]} {nb_tuple_delta_r_x_e_y}\n' + contraintes_r_x_e_y
    contrainte_Delta_r_x_r_y = f'2 {interference["x"]+nb_stations} {interference["y"]+nb_stations} {interference["Delta"]} {nb_tuple_delta_r_x_r_y}\n' + contraintes_r_x_r_y

    contraintes_Delta+= contrainte_Delta_e_x_e_y
    contraintes_Delta+= contrainte_Delta_e_x_r_y
    contraintes_Delta+= contrainte_Delta_r_x_e_y
    contraintes_Delta+= contrainte_Delta_r_x_r_y
    nb_fct_cout+=4

#### Contraintes sur les liaisons (par extension, contraintes fortes)

On vérifie que si deux stations sont liées, alors $E_i = R_j$ et $R_i = E_j$. 

In [11]:
contraintes_liaisons = f''
for liaison in liaisons:
    fqc_e_x = stations[liaison["x"]]['emetteur']
    fqc_r_x = stations[liaison["x"]]['recepteur']
    fqc_e_y = stations[liaison["y"]]['emetteur']
    fqc_r_y = stations[liaison["y"]]['recepteur']


    fqc_possible_e_x_r_y = list(product(fqc_e_x, fqc_r_y))
    fqc_possible_r_x_e_y = list(product(fqc_r_x, fqc_e_y))

    contraintes_e_x_r_y = f''
    nb_tuple_liaison_e_x_r_y = 0
    for item in fqc_possible:
        if item[0] == item[1]:
            contraintes_e_x_r_y += f'{item[0]} {item[1]} 0\n'
            nb_tuple_liaison_e_x_r_y +=1  

    contraintes_r_x_e_y = f''
    nb_tuple_liaison_r_x_e_y = 0
    for item in fqc_possible:
        if item[0] == item[1]:
            contraintes_r_x_e_y += f'{item[0]} {item[1]} 0\n'
            nb_tuple_liaison_r_x_e_y +=1  


    contraintes_liaisons += f'2 {liaison["x"]+nb_stations} {liaison["y"]} {max_bound} {nb_tuple_liaison_e_x_r_y}\n' + contraintes_e_x_r_y
    contraintes_liaisons += f'2 {liaison["x"]} {liaison["y"]+nb_stations} {max_bound} {nb_tuple_liaison_r_x_e_y}\n' + contraintes_r_x_e_y
    nb_fct_cout+=2

## Écriture du fichier WCSP

In [13]:
fichier = open("allocation_fqc.wcsp", 'w')
fichier.write(f'frequence_allocation {2*nb_stations} {fqc_max} {nb_fct_cout} {max_bound}\n' + taille_domaine + contraintes_fqc + contraintes_delta + contraintes_Delta + contraintes_liaisons)
fichier.close()